# 1 - Install missing dependencies

For the plotting of the Land Cover map we need the earthpy python package, that relies on the libspatialindex-dev lib.

In [0]:
!apt-get install libspatialindex-dev
!pip install --upgrade earthpy

# 2 - Mount Google Drive 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# 3 - Import necessary python libraries

In [0]:
import numpy as np
import rasterio as rio
from rasterio.windows import Window

import earthpy.plot as ep
import earthpy.spatial as es

from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

# 4 - Read the data

We read all bands into a dictionary, that hold the band names as key and the respective array as value. All data is cut to the same extent of 3000 x 3000 pixels.


In [0]:
list_of_layers = ['VV_sigma0', 'VH_sigma0', 
                  'VV_gamma0flat', 'VH_gamma0flat',
                  'layover', 'shadow', ]

# this refers to the version we produced during data generation, i.e. volume model and buffer
data_version = 'SRTMGL1_003_volume_buf_0'

# we read all layers into a dictionary 
dataDict = {}
for layer in list_of_layers:
    with rio.open('/content/drive/My Drive/slope_correction {}/{}.tif'.format(data_version, layer)) as src:
        print('Reading ' + layer)
        dataDict[layer] = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]
        print(dataDict[layer].shape)

# read the corrected data from surface model
with rio.open('/content/drive/My Drive/slope_correction SRTMGL1_003_surface_buf_0/VV_gamma0flat.tif') as src:
  VV_gamma0_direct = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]  

with rio.open('/content/drive/My Drive/slope_correction SRTMGL1_003_surface_buf_0/VH_gamma0flat.tif') as src:
  VH_gamma0_direct = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]  

# 5 - Prepare data for plotting

We create 3 3-D array for the RGB plotting of the Sentinel-1 data (original, surface and volume). Furthermore, the layover and shadow masks are inverted for plotting.


In [0]:
# create empty 3d arrays
arr_orig = np.empty((3, dataDict['VV_sigma0'].shape[0], dataDict['VV_sigma0'].shape[1]))
arr_vol = arr_orig.copy()
arr_dir = arr_orig.copy()

# create sigma0 rgb array
arr_orig[0] = dataDict['VV_sigma0'].copy()
arr_orig[1] = dataDict['VH_sigma0'].copy()
arr_orig[2] = arr_orig[0] - arr_orig[1]

# create gamma0flat rgb array of volume model
arr_vol[0] = dataDict['VV_gamma0flat'].copy()
arr_vol[1] = dataDict['VH_gamma0flat'].copy()
arr_vol[2] = arr_vol[0] - arr_vol[1]

# create gamma0flat rgb array of direct model
arr_dir[0] = VV_gamma0_direct
arr_dir[1] = VH_gamma0_direct
arr_dir[2] = arr_dir[0] - arr_dir[1]

# create difference array
arr_diff = np.subtract(arr_vol[0], arr_dir[0])

# get layover and invert
layover = dataDict['layover'].astype('float32').copy()
layover[layover == 1] = np.nan
layover[layover == 0] = 1


# get shadow and invert
shadow = dataDict['shadow'].astype('float32').copy()
shadow[shadow == 1] = np.nan
shadow[shadow == 0] = 1


# 6 - Plot figure

In [0]:
# initialize figure
f, ax = plt.subplots(2, 2, sharey=True, sharex=True, figsize=(20, 20))

# force nn interpolation for figure
interpolation_method = 'nearest'

# plot sentinel-1 sigma0
ep.plot_rgb(arr_orig, stretch=True, str_clip=0.5, ax=ax[0, 0])

# plot sentinel-1 gamma0flat with layover and shadow
img1 = ep.plot_rgb(arr_vol, stretch=True, str_clip=0.5, ax=ax[0, 1])
img1 = ax[0, 1].imshow(layover, cmap=plt.cm.Greys, interpolation=interpolation_method)
img1 = ax[0, 1].imshow(shadow, cmap=plt.cm.Greys_r, interpolation=interpolation_method)
ep.draw_legend(im_ax=img1, bbox=(0,1,0,0),  titles=['Shadow', 'Layover'], cmap=plt.cm.bwr, classes=[0, 1])

img2 = ep.plot_rgb(arr_dir, stretch=True, str_clip=0.5, ax=ax[1, 0])
img2 = ax[1, 0].imshow(layover, cmap=plt.cm.Greys, interpolation=interpolation_method)
img2 = ax[1, 0].imshow(shadow, cmap=plt.cm.Greys_r, interpolation=interpolation_method)
ep.draw_legend(im_ax=img2, bbox=(0,1,0,0),  titles=['Shadow', 'Layover'], cmap=plt.cm.bwr, classes=[0, 1])

img3 = ep.plot_bands(arr_diff, scale=False, vmin=-5, vmax=5, ax=ax[1, 1], cbar=False)
img3 = ax[1, 1].imshow(layover, cmap=plt.cm.bwr_r, interpolation=interpolation_method)
img3 = ax[1, 1].imshow(shadow, cmap=plt.cm.bwr, interpolation=interpolation_method)
ep.draw_legend(im_ax=img3, bbox=(0,1,0,0),  titles=['Shadow', 'Layover'], cmap=plt.cm.bwr, classes=[0, 1])

plt.tight_layout()
plt.savefig('/content/drive/My Drive/slope_correction/Figure_4.png')